In [1]:
import string
import pke
import json
import numpy as np
from nltk.corpus import stopwords
from matplotlib import pyplot as plt

## Change the file path to your current path

In [ ]:
# Here is a sample of how to read data and use the dataset
import json
f = open('./test.json')
data = json.load(f)
f.close()
# data is a dictionary with all of the data 
for count, body in data.items():
    abstract    = body['abstract']
    title       = body['title']
    kwds_in     = body['keywords_in']
    kwds_not_in = body['keywords_not_in']

In [2]:
# a simple evaluation method that matches the exact string
def match(list1, list2):
    same = set(list1).intersection(set(list2))
    return len(same)

In [3]:
# load the data
f = open('/home/ubuntu/pubmed/test.json')
total = json.load(f)
f.close()

In [19]:
# the result array [@5, @10, @15] stores the result of extracting 5, 10 and 15 keyphrases 
yake_ground_truth = np.zeros(3)
yake_matched = np.zeros(3)
yake_extracted = np.zeros(3)

# write to file store kwds for extractive methods as a backup 
f = open('/home/ubuntu/pubmed/extracted_kwds/yakekwds.json', 'w')
final = {}

# begin extraction
for count, body in total.items():
    abstract = body['abstract']
    kwds_in = body['keywords_in']
    kwds_not_in = body['keywords_not_in']
    # pke method setup
    extractor = pke.unsupervised.YAKE()
    extractor.load_document(input=abstract,
                            language='en',
                            normalization=None)
    stoplist = stopwords.words('english')
    extractor.candidate_selection(n=3, stoplist=stoplist)
    window = 2
    use_stems = False # use stems instead of words for weighting
    extractor.candidate_weighting(window=window,
                                  stoplist=stoplist,
                                  use_stems=use_stems)

    threshold = 0.8
    temp = {5:[], 10:[], 15:[]}
    # extract top 5, 10, 15 words
    for nn in [5,10,15]:
        keyphrases = extractor.get_n_best(n=nn, threshold=threshold)
        keyphrases = [i[0] for i in keyphrases]
        index = (int)(nn/5-1)
        yake_extracted[index] += nn
        yake_ground_truth[index] += len(kwds_in)
        yake_matched[index] += match(kwds_in, keyphrases)
        temp[nn] = keyphrases
    final[count] = temp
# write to file
json.dump(final, f, indent = 4)
f.close()

# calculate result
yake_precision = yake_matched / yake_extracted
yake_recall = yake_matched / yake_ground_truth
yake_f1 = 2 * yake_precision * yake_recall / (yake_precision + yake_recall)



In [11]:
# write result to a file
f = open('/home/ubuntu/pubmed/acc.txt','a')
f.write(f'yake_precision: {yake_precision}\n')
f.write(f'yake_recall: {yake_recall}\n')
f.write(f'yake_f1: {yake_f1}\n')
f.close()

## Below are just the same process repeated with other methods in the pke package

In [ ]:
textrank_ground_truth = np.zeros(3)
textrank_matched = np.zeros(3)
textrank_extracted = np.zeros(3)

f = open('/home/ubuntu/pubmed/extracted_kwds/textrankkwds.json', 'w')
final = {}

for count, body in total.items():
    abstract = body['abstract']
    kwds_in = body['keywords_in']
    kwds_not_in = body['keywords_not_in']
    if abstract == "":
        continue
    if kwds_in == [] and kwds_not_in == []:
        continue
        
    pos = {'NOUN', 'PROPN', 'ADJ'}

    extractor = pke.unsupervised.TextRank()

    extractor.load_document(input=abstract,
                            language='en',
                            normalization=None)

    extractor.candidate_weighting(window=2,
                                  pos=pos,
                                  top_percent=0.33)
    temp = {5:[], 10:[], 15:[]}
    for nn in [5,10,15]:
        keyphrases = extractor.get_n_best(n=nn)
        keyphrases = [i[0] for i in keyphrases]
        index = (int)(nn/5-1)
        textrank_extracted[index] += nn
        textrank_ground_truth[index] += len(kwds_in)
        textrank_matched[index] += match(kwds_in, keyphrases)
        temp[nn] = keyphrases
    final[count] = temp

json.dump(final, f, indent = 4)
f.close()
        
textrank_precision = textrank_matched / textrank_extracted
textrank_recall = textrank_matched / textrank_ground_truth
textrank_f1 = 2 * textrank_precision * textrank_recall / (textrank_precision + textrank_recall)


In [ ]:
f = open('/home/ubuntu/pubmed/acc.txt','a')
f.write(f'textrank_precision: {textrank_precision}\n')
f.write(f'textrank_recall: {textrank_recall}\n')
f.write(f'textrank_f1: {textrank_f1}\n')
f.close()

In [21]:
singlerank_ground_truth = np.zeros(3)
singlerank_matched = np.zeros(3)
singlerank_extracted = np.zeros(3)

f = open('/home/ubuntu/pubmed/extracted_kwds/singlerankkwds.json', 'w')
final = {}

for count, body in total.items():
    abstract = body['abstract']
    kwds_in = body['keywords_in']
    kwds_not_in = body['keywords_not_in']
    if abstract == "":
        continue
    if kwds_in == [] and kwds_not_in == []:
        continue
        
    pos = {'NOUN', 'PROPN', 'ADJ'}

    extractor = pke.unsupervised.SingleRank()

    extractor.load_document(input=abstract,
                            language='en',
                            normalization=None)

    extractor.candidate_selection(pos=pos)

    extractor.candidate_weighting(window=10, pos=pos)
    temp = {5:[], 10:[], 15:[]}
    for nn in [5,10,15]:
        keyphrases = extractor.get_n_best(n=nn)
        keyphrases = [i[0] for i in keyphrases]
        index = (int)(nn/5-1)
        singlerank_extracted[index] += nn
        singlerank_ground_truth[index] += len(kwds_in)
        singlerank_matched[index] += match(kwds_in, keyphrases)
        temp[nn] = keyphrases
    final[count] = temp

json.dump(final, f, indent = 4)
f.close()
                
singlerank_precision = singlerank_matched / singlerank_extracted
singlerank_recall = singlerank_matched / singlerank_ground_truth
singlerank_f1 = 2 * singlerank_precision * singlerank_recall / (singlerank_precision + singlerank_recall)


In [13]:
f = open('/home/ubuntu/pubmed/acc.txt','a')
f.write(f'singlerank_precision: {singlerank_precision}\n')
f.write(f'singlerank_recall: {singlerank_recall}\n')
f.write(f'singlerank_f1: {singlerank_f1}\n')
f.close()

In [23]:
topicrank_ground_truth = np.zeros(3)
topicrank_matched = np.zeros(3)
topicrank_extracted = np.zeros(3)

f = open('/home/ubuntu/pubmed/extracted_kwds/topicrankkwds.json', 'w')
final = {}

for count, body in total.items():
    abstract = body['abstract']
    kwds_in = body['keywords_in']
    kwds_not_in = body['keywords_not_in']
    if abstract == "":
        continue
    if kwds_in == [] and kwds_not_in == []:
        continue
        
    extractor = pke.unsupervised.TopicRank()
    extractor.load_document(input=abstract)

    pos = {'NOUN', 'PROPN', 'ADJ'}
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')
    extractor.candidate_selection(pos=pos, stoplist=stoplist)

    extractor.candidate_weighting(threshold=0.74, method='average')
    temp = {5:[], 10:[], 15:[]}
    for nn in [5,10,15]:
        keyphrases = extractor.get_n_best(n=nn)
        keyphrases = [i[0] for i in keyphrases]
        index = (int)(nn/5-1)
        topicrank_extracted[index] += nn
        topicrank_ground_truth[index] += len(kwds_in)
        topicrank_matched[index] += match(kwds_in, keyphrases)
        temp[nn] = keyphrases
    final[count] = temp

json.dump(final, f, indent = 4)
f.close()        
topicrank_precision = topicrank_matched / topicrank_extracted
topicrank_recall = topicrank_matched / topicrank_ground_truth
topicrank_f1 = 2 * topicrank_precision * topicrank_recall / (topicrank_precision + topicrank_recall)


In [ ]:
f = open('/home/ubuntu/pubmed/acc.txt','a')
f.write(f'topicrank_precision: {topicrank_precision}\n')
f.write(f'topicrank_recall: {topicrank_recall}\n')
f.write(f'topicrank_f1: {topicrank_f1}\n')
f.close()

In [24]:
positionrank_ground_truth = np.zeros(3)
positionrank_matched = np.zeros(3)
positionrank_extracted = np.zeros(3)

f = open('/home/ubuntu/pubmed/extracted_kwds/positionrankkwds.json', 'w')
final = {}

for count, body in total.items():
    abstract = body['abstract']
    kwds_in = body['keywords_in']
    kwds_not_in = body['keywords_not_in']
    if abstract == "":
        continue
    if kwds_in == [] and kwds_not_in == []:
        continue
    pos = {'NOUN', 'PROPN', 'ADJ'}

    grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

    extractor = pke.unsupervised.PositionRank()
    extractor.load_document(input=abstract,
                            language='en',
                            normalization=None)

    extractor.candidate_selection(grammar=grammar,
                                  maximum_word_number=3)

    extractor.candidate_weighting(window=10, pos=pos)
    temp = {5:[], 10:[], 15:[]}
    for nn in [5,10,15]:
        keyphrases = extractor.get_n_best(n=nn)
        keyphrases = [i[0] for i in keyphrases]
        index = (int)(nn/5-1)
        positionrank_extracted[index] += nn
        positionrank_ground_truth[index] += len(kwds_in)
        positionrank_matched[index] += match(kwds_in, keyphrases)
        temp[nn] = keyphrases
    final[count] = temp

json.dump(final, f, indent = 4)
f.close()          
positionrank_precision = positionrank_matched / positionrank_extracted
positionrank_recall = positionrank_matched / positionrank_ground_truth
positionrank_f1 = 2 * positionrank_precision * positionrank_recall / (positionrank_precision + positionrank_recall)


In [19]:
f = open('/home/ubuntu/pubmed/acc.txt','a')
f.write(f'positionrank_precision: {positionrank_precision}\n')
f.write(f'positionrank_recall: {positionrank_recall}\n')
f.write(f'positionrank_f1: {positionrank_f1}\n')
f.close()

In [25]:
multirank_ground_truth = np.zeros(3)
multirank_matched = np.zeros(3)
multirank_extracted = np.zeros(3)

f = open('/home/ubuntu/pubmed/extracted_kwds/multirankkwds.json', 'w')
final = {}

for count, body in total.items():
    abstract = body['abstract']
    kwds_in = body['keywords_in']
    kwds_not_in = body['keywords_not_in']
    if abstract == "":
        continue
    if kwds_in == [] and kwds_not_in == []:
        continue
    extractor = pke.unsupervised.MultipartiteRank()

    extractor.load_document(input=abstract)

    pos = {'NOUN', 'PROPN', 'ADJ'}
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')
    extractor.candidate_selection(pos=pos, stoplist=stoplist)

    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.74,
                                  method='average')
    temp = {5:[], 10:[], 15:[]}
    for nn in [5,10,15]:
        keyphrases = extractor.get_n_best(n=nn)
        keyphrases = [i[0] for i in keyphrases]
        index = (int)(nn/5-1)
        multirank_extracted[index] += nn
        multirank_ground_truth[index] += len(kwds_in)
        multirank_matched[index] += match(kwds_in, keyphrases)
        temp[nn] = keyphrases
    final[count] = temp

json.dump(final, f, indent = 4)
f.close()                  
multirank_precision = multirank_matched / multirank_extracted
multirank_recall = multirank_matched / multirank_ground_truth
multirank_f1 = 2 * multirank_precision * multirank_recall / (multirank_precision + multirank_recall)


In [ ]:
f = open('/home/ubuntu/pubmed/acc.txt','a')
f.write(f'multirank_precision: {multirank_precision}\n')
f.write(f'multirank_recall: {multirank_recall}\n')
f.write(f'multirank_f1: {multirank_f1}\n')
f.close()